In [1]:
import sys
import usb.core
import usb.util
import usb
import numpy as np

VID = 0x104d
PID = 0x4000

dev = usb.core.find(idVendor=VID,idProduct=PID)

if dev is None:
    raise ValueError('Device not found.')
else:
    print('Device found.')
   
      
dev.set_configuration()   
cfg = dev.get_active_configuration()
intf = cfg[(0,0)]

print( usb.util.get_string( dev, dev.iSerialNumber ) )

Device found.
12345678


In [2]:
#print(dev)
for i in dev:
    sys.stdout.write(str(cfg.bMaxPower)+ '\n')

250


In [3]:
# show config / interface / endpoints:
for config in dev:
    sys.stdout.write('Config ' + str(config.bConfigurationValue) + '\n')
    for interface in config:
        sys.stdout.write('\t' + \
            'Interface ' + str(interface.bInterfaceNumber) + \
                         ',' + \
                         str(interface.bAlternateSetting) + \
                         '\n')
        for endpoint in interface:
            sys.stdout.write('\t\t' + \
                             'Endpoint ' + str(endpoint.bEndpointAddress) + \
                             '\n')

Config 1
	Interface 0,0
		Endpoint 129
		Endpoint 2


In [4]:
ep_out = usb.util.find_descriptor(
    intf,
    # match the first OUT endpoint
    custom_match = \
    lambda e: \
        usb.util.endpoint_direction(e.bEndpointAddress) == \
        usb.util.ENDPOINT_OUT)

assert ep_out is not None

print(ep_out)

ep_in = usb.util.find_descriptor(
    intf,
    # match the first IN endpoint
    custom_match = \
    lambda e: \
        usb.util.endpoint_direction(e.bEndpointAddress) == \
        usb.util.ENDPOINT_IN)

print(ep_in)

assert ep_in is not None
assert ep_in.wMaxPacketSize == 64

      ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x0
      ENDPOINT 0x81: Bulk IN ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x81 IN
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x0


In [5]:
# write the command to find ID


# ASCII codes converted from text
#command = [42, 73, 68, 78, 63, 13]
command = b"*IDN?\r"

ep_out.write(command)


data = ep_in.read(74,timeout=1000);

# Convert ASCII codes back to text
data_ = ""
for char in data:
    data_ = data_ + chr(char)
    
print(data_)

New_Focus 8742 v3.04 09/09/16 53365



In [29]:
command = b"1>2PA+1505\r"
ep_out.write(command, timeout=1000)

11

In [30]:
command = b"1>2PA?\r"
ep_out.write(command, timeout=1000)
# write(self, data, timeout = None): Writes data to the endpoint. 
# The parameter data contains the data to be sent to the endpoint and timeout is the time limit of the operation.
# The transfer type and endpoint address are automatically inferred.
# The method returns the number of bytes written.
data = ep_in.read(64)
# read(self, size_or_buffer, timeout = None): Reads data from the endpoint.
# The parameter size_or_buffer is either the number of bytes to read or an array object where the data will
# be put in and timeout is the time limit of the operation. The transfer type and endpoint address are 
# automatically inferred.
# The method returns either an array object or the number of bytes actually read.

data_ = ""
for char in data:
    data_ = data_ + chr(char)
    
print(data_)

1>0

